# Process the original file to match function

In [ ]:
# to auto-reload the imports
# if we change something in our functions
%load_ext autoreload
%autoreload 2

%load_ext nb_black

# import the libraries required to do the work
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
import pandas as pd
import seaborn as sns
import datetime
from scipy.stats import norm
from scipy.interpolate import interp1d
import os
from tqdm.notebook import tqdm
import re

from src.utils.sharepoint import get_T1_ren_6kPax_schedule
from decouple import AutoConfig
from pathlib import Path

In [ ]:
# get the schedule from Sharepoint
# get_T1_ren_6kPax_schedule()

# get the paths to config (could be made as a function for notebooks)

DOTENV_FILE_PATH = Path(os.getcwd()) / "../../../data/secret/.env"
config = AutoConfig(search_path=DOTENV_FILE_PATH)

path_relative = config("T1_ren_6kPax_schedule_path")

path_data = Path(os.getcwd()) / ".." / ".." / ".." / path_relative

In [ ]:
# define a function to insert space in flight number
def insert_space_after_letters(test_str: str):
    res = re.sub("[A-Za-z]+", lambda ele: ele[0] + " ", test_str)
    return res

In [ ]:
# import the schedule from the excel file produced by Aero department
data_arr = pd.read_excel(
    path_data,
    header=0,
    sheet_name="ARR",
)

data_dep = pd.read_excel(
    path_data,
    header=0,
    sheet_name="DEP",
)

In [ ]:
# edit data to match application format
data_arr_processed = data_arr.copy()
data_arr_processed["A/D"] = "A"
data_arr_processed["T1/T2(MM/9C/7C/TW)"] = "T1"
data_arr_processed["Int'l Regions"] = "unknown"

dct_name_change = {
    "貨客区分": "Category(P/C/O)",
    "DI区分": "Sector",
    "航空機識別": "Flight Number",  # <- to be split with space between letters and digits
    "座席数": "SEATS FC",
    "搭乗者数": "Pax_SUM FC",
    "到着予定日": "Flight Date",
    "STA": "Scheduled Time",
}

data_arr_processed.rename(columns=dct_name_change, inplace=True)

data_arr_processed["Flight Number"] = data_arr_processed["Flight Number"].apply(
    insert_space_after_letters
)

In [ ]:
# edit data to match application format
data_dep_processed = data_dep.copy()
data_dep_processed["A/D"] = "D"
data_dep_processed["T1/T2(MM/9C/7C/TW)"] = "T1"
data_dep_processed["Intl Regions"] = "unknown"

dct_name_change = {
    "貨客区分": "Category(P/C/O)",
    "DI区分": "Sector",
    "航空機識別": "Flight Number",  # <- to be split with space between letters and digits
    "座席数": "SEATS FC",
    "搭乗者数": "PAX_SUM FC",
    "出発予定日": "Flight Date",
    "STD": "Scheduled Time",
}

data_dep_processed.rename(columns=dct_name_change, inplace=True)

data_dep_processed["Flight Number"] = data_dep_processed["Flight Number"].apply(
    insert_space_after_letters
)

In [ ]:
# keep only relevant columns
data_dep_processed = data_dep_processed[
    [
        "A/D",
        "T1/T2(MM/9C/7C/TW)",
        "Intl Regions",
        "Category(P/C/O)",
        "Sector",
        "Flight Number",
        "SEATS FC",
        "PAX_SUM FC",
        "Flight Date",
        "Scheduled Time",
    ]
]

# correct input mistake (?)
mask_replace = data_dep_processed["PAX_SUM FC"] == "-"
data_dep_processed.loc[mask_replace, "PAX_SUM FC"] = 0

mask_replace = data_dep_processed["SEATS FC"] == "-"
data_dep_processed.loc[mask_replace, "SEATS FC"] = 0

# add seat number for "NEW" flights
mask_replace = (
    data_dep_processed["Flight Number"].str.split(" ", 1, expand=True)[0] == "NEW"
) & (data_dep_processed["PAX_SUM FC"] > 148)

data_dep_processed.loc[mask_replace,"SEATS FC"] = 285

mask_replace = (
    data_dep_processed["Flight Number"].str.split(" ", 1, expand=True)[0] == "NEW"
) & (data_dep_processed["PAX_SUM FC"] <= 148)

data_dep_processed.loc[mask_replace,"SEATS FC"] = 180

In [ ]:
test = data_dep_processed["Flight Number"].apply(lambda x:x.split(" ")[0])

In [ ]:
# we should add a step to merge the two in one
# as we do not have seats and Pax for T1 schedule, let's forget about it for now

In [ ]:
output_path = (
    Path(os.getcwd())
    / "../../../data/processed/Schedule (30th terminal peak, 6000 pax)_PROCESSED.xlsx"
)

writer = pd.ExcelWriter(
    output_path,
)

with writer as writer:
    data_dep_processed.to_excel(writer, sheet_name="schedule")